# LLM API Testing

# Get response from LLM

In [1]:
import json
import datetime

# Set your OpenAI API key
import os
import openai

api_key = os.environ.get("OPENAI_KEY")



In [3]:
user_prompt = "instructions_v4.txt"

# Define a function to get a response from the OpenAI API
def get_openai_response_fake(user_prompt, prompt):
    # use user_prompt as file_name to get the full game instructions
    try:
        prompt = {k:v for k,v in prompt.items() if v}  # Remove empty values from prompt (e.g., initial prompt)
        input = {
            'user_prompt': user_prompt,    # Full game instructions
            'prompt': json.dumps(prompt),  # Current move instructions (dict dumped to string)
        }

        # Todo: actually call API
        response = "2 1022"  # Hardcoded fake response
        output = input.copy()
        output['current_shape'] = prompt.get('current_shape')
        output['last_shape'] = prompt.get('last_shape')
        output['response'] = response
        output['timestamp'] = datetime.datetime.now().isoformat()
        output['valid_move'] = True # TODO Hardcoded, but need to write a function for checking
        return output
    except Exception as e:
        return f"An error occurred: {e}"
    
# Valid moves example
# 1023 -> 2 1022 -> 4 1020 512

# Single example
user_prompt = "instructions/version1.txt"
prompt={
    "current_shape": "1023", 
    "last_shape": ""
}
get_openai_response_fake(user_prompt, prompt)

{'user_prompt': 'instructions/version1.txt',
 'prompt': '{"current_shape": "1023"}',
 'current_shape': '1023',
 'last_shape': None,
 'response': '2 1022',
 'timestamp': '2025-01-30T14:03:08.350019',
 'valid_move': True}

In [4]:
# Interaction loop for game

THRESHOLD_TOTAL_MOVES = 10
THRESHOLD_TOTAL_RETRIES = 10

user_prompt = "instructions/version1.txt"
prompt={
    "current_shape": "1023", 
    "last_shape": ""
}

move_count = 0
retry_count = 0

while move_count < THRESHOLD_TOTAL_MOVES and retry_count < THRESHOLD_TOTAL_RETRIES:
    output = get_openai_response_fake(user_prompt, prompt)
    print(output)
    if output['valid_move']:
        move_count += 1
        retry_count = 0
        prompt['current_shape'] = output['response']
    else:
        retry_count += 1
        print(f"Invalid move, retrying... {retry_count} / {THRESHOLD_TOTAL_RETRIES}")
    
    # TODO: save response dictionary to file (append as you go)
    with open('data/responses.json', 'a') as f:
        f.write(json.dumps(output) + '\n')


{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "1023"}', 'current_shape': '1023', 'last_shape': None, 'response': '2 1022', 'timestamp': '2025-01-30T14:03:08.384481', 'valid_move': True}
{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "2 1022"}', 'current_shape': '2 1022', 'last_shape': None, 'response': '2 1022', 'timestamp': '2025-01-30T14:03:08.384932', 'valid_move': True}
{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "2 1022"}', 'current_shape': '2 1022', 'last_shape': None, 'response': '2 1022', 'timestamp': '2025-01-30T14:03:08.385012', 'valid_move': True}
{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "2 1022"}', 'current_shape': '2 1022', 'last_shape': None, 'response': '2 1022', 'timestamp': '2025-01-30T14:03:08.385114', 'valid_move': True}
{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "2 1022"}', 'current_shape': '2 1022', 'last_shape': None,

In [2]:
import json
import datetime
import os
import openai

from functions.tools import is_valid_move

# Set your OpenAI API key
api_key = os.environ.get('OPENAI_KEY')
print(api_key)

None


In [ ]:


# Define a function to get a response from the OpenAI API
def get_openai_response(user_prompt, prompt):
    try:
        # Remove empty values from prompt (e.g., initial prompt)
        prompt = {k: v for k, v in prompt.items() if v}
        input_data = {
            'user_prompt': user_prompt,    # Full game instructions
            'prompt': json.dumps(prompt),  # Current move instructions (dict dumped to string)
        }

        # Make the API call
        response = openai.chat.completions.create(
            model="gpt-o3-mini",  # Specify the engine you want to use
            prompt=input_data['prompt'],
            max_tokens=100  # Adjust the number of tokens as needed
        )

        # Process the API response
        output = input_data.copy()
        output['current_shape'] = prompt.get('current_shape')
        output['last_shape'] = prompt.get('last_shape')
        output['response'] = response.choices[0].text.strip()
        output['timestamp'] = datetime.datetime.now().isoformat()
        output['valid_move'] = is_valid_move(output['response'])  # Check if the move is valid
        return output
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
user_prompt = "instructions_v4.txt"
prompt = {
    "current_shape": "shape1",
    "last_shape": "shape0",
    "move": "move1"
}

response = get_openai_response(user_prompt, prompt)
print(response)

An error occurred: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
